# CHAPTER 2.2

### Creating a multi-class classifier to play rock paper scissors

We will use the Rock-Paper-Scissors Images dataset, which is hosted on Kaggle at the following location: https://www.kaggle.com/drgfreeman/rockpaperscissors.

We will train a multi-class CNN to distinguish between the three classes of the Rock Paper Scissors game

In [1]:
import os
import pathlib

import glob
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras.losses import CategoricalCrossentropy

In [2]:
# CLASSES should be same with the file name 
CLASSES = ['rock', 'paper', 'scissors']
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [19]:
def load_image_and_label(image_path, target_size=(32, 32)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.convert_image_dtype(image, np.float32)
    image = tf.image.resize(image, target_size)

    label = tf.strings.split(image_path, os.path.sep)[-2]
    label = (label == CLASSES)  # One-hot encode.
    label = tf.dtypes.cast(label, tf.float32)

    return image, label

!!! NEXT CELLS ARE WRITTEN FOR SHOWING THE DETAILS ABOUT load_image_and_label function!!!

In [7]:
img_pth=r'C:\Users\Zeki\.keras\datasets\rockpaperscissors\rps-cv-images\scissors\0Flw60Z2MAWWKn6S.png'
label = tf.strings.split(img_pth, os.path.sep)[-2]

In [8]:
label

<tf.Tensor: shape=(), dtype=string, numpy=b'scissors'>

In [9]:
print(label)

tf.Tensor(b'scissors', shape=(), dtype=string)


In [12]:
label.numpy()

b'scissors'

In [13]:
label = (label == CLASSES)  # One-hot encode.


In [14]:
label

<tf.Tensor: shape=(3,), dtype=bool, numpy=array([False, False,  True])>

In [15]:
label = tf.dtypes.cast(label, tf.float32)

In [16]:
label

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 0., 1.], dtype=float32)>

In [18]:
label.numpy()

array([0., 0., 1.], dtype=float32)

In [20]:
img = tf.io.read_file(img_pth)

In [25]:
img = tf.image.decode_jpeg(img, channels=3)
    

In [26]:
img

<tf.Tensor: shape=(200, 300, 3), dtype=uint8, numpy=
array([[[ 37, 115,  28],
        [ 40, 117,  30],
        [ 40, 117,  30],
        ...,
        [ 89, 102,  75],
        [ 91, 104,  77],
        [ 90, 102,  67]],

       [[ 36, 114,  24],
        [ 40, 118,  28],
        [ 40, 118,  28],
        ...,
        [ 47, 126,  27],
        [ 48, 127,  28],
        [ 46, 128,  28]],

       [[ 36, 114,  24],
        [ 39, 116,  27],
        [ 39, 116,  27],
        ...,
        [ 44, 124,  24],
        [ 48, 127,  28],
        [ 46, 128,  28]],

       ...,

       [[ 36, 117,  21],
        [ 37, 118,  23],
        [ 37, 118,  23],
        ...,
        [ 42, 121,  24],
        [ 41, 120,  23],
        [ 40, 119,  22]],

       [[ 38, 119,  24],
        [ 38, 119,  24],
        [ 39, 120,  25],
        ...,
        [ 41, 120,  23],
        [ 41, 120,  23],
        [ 41, 120,  23]],

       [[ 37, 118,  21],
        [ 39, 120,  25],
        [ 39, 120,  25],
        ...,
        [ 41, 120,  2

In [27]:
img = tf.image.rgb_to_grayscale(img)
    

In [28]:
img

<tf.Tensor: shape=(200, 300, 1), dtype=uint8, numpy=
array([[[81],
        [84],
        [84],
        ...,
        [95],
        [97],
        [94]],

       [[80],
        [84],
        [84],
        ...,
        [91],
        [92],
        [92]],

       [[80],
        [82],
        [82],
        ...,
        [88],
        [92],
        [92]],

       ...,

       [[81],
        [83],
        [83],
        ...,
        [86],
        [85],
        [84]],

       [[84],
        [84],
        [85],
        ...,
        [85],
        [85],
        [85]],

       [[82],
        [85],
        [85],
        ...,
        [85],
        [84],
        [85]]], dtype=uint8)>

In [29]:
img = tf.image.convert_image_dtype(img, np.float32)
    

In [30]:
img

<tf.Tensor: shape=(200, 300, 1), dtype=float32, numpy=
array([[[0.31764707],
        [0.32941177],
        [0.32941177],
        ...,
        [0.37254903],
        [0.3803922 ],
        [0.36862746]],

       [[0.3137255 ],
        [0.32941177],
        [0.32941177],
        ...,
        [0.35686275],
        [0.36078432],
        [0.36078432]],

       [[0.3137255 ],
        [0.32156864],
        [0.32156864],
        ...,
        [0.34509805],
        [0.36078432],
        [0.36078432]],

       ...,

       [[0.31764707],
        [0.3254902 ],
        [0.3254902 ],
        ...,
        [0.3372549 ],
        [0.33333334],
        [0.32941177]],

       [[0.32941177],
        [0.32941177],
        [0.33333334],
        ...,
        [0.33333334],
        [0.33333334],
        [0.33333334]],

       [[0.32156864],
        [0.33333334],
        [0.33333334],
        ...,
        [0.33333334],
        [0.32941177],
        [0.33333334]]], dtype=float32)>

In [31]:
img = tf.image.resize(img, (32,32))

In [32]:
img

<tf.Tensor: shape=(32, 32, 1), dtype=float32, numpy=
array([[[0.32150736],
        [0.33440566],
        [0.342739  ],
        ...,
        [0.3599571 ],
        [0.36351103],
        [0.35612747]],

       [[0.32941177],
        [0.3335478 ],
        [0.34779412],
        ...,
        [0.36102942],
        [0.36427698],
        [0.3630821 ]],

       [[0.33406863],
        [0.33520222],
        [0.33936888],
        ...,
        [0.36124387],
        [0.3629902 ],
        [0.35686275]],

       ...,

       [[0.30940565],
        [0.3137255 ],
        [0.32637867],
        ...,
        [0.3335478 ],
        [0.33036155],
        [0.32941177]],

       [[0.31029412],
        [0.31400123],
        [0.3259804 ],
        ...,
        [0.3409314 ],
        [0.33360907],
        [0.3372549 ]],

       [[0.32748163],
        [0.31075367],
        [0.32273287],
        ...,
        [0.33357844],
        [0.33195466],
        [0.3372549 ]]], dtype=float32)>

In [33]:
def build_network():
    input_layer = Input(shape=(32, 32, 1))
    x = Conv2D(filters=32,
               kernel_size=(3, 3),
               padding='same',
               strides=(1, 1))(input_layer)
    x = ReLU()(x)
    x = Dropout(rate=0.5)(x)

    x = Flatten()(x)
    x = Dense(units=3)(x)
    output = Softmax()(x)

    model = Model(inputs=input_layer, outputs=output)
    return model

In [34]:
def prepare_dataset(dataset_path, buffer_size, batch_size,shuffle=True):
    dataset = (tf.data.Dataset.from_tensor_slices(dataset_path).map(load_image_and_label,
                    num_parallel_calls=AUTOTUNE))

    if shuffle:
        dataset.shuffle(buffer_size=buffer_size)

    dataset = (dataset.batch(batch_size=batch_size).prefetch(buffer_size=buffer_size))

    return dataset

In [35]:
file_patten = (pathlib.Path.home() / '.keras' / 'datasets' /
               'rockpaperscissors' / 'rps-cv-images' / '*' /
               '*.png')
file_pattern = str(file_patten)
dataset_paths = [*glob.glob(file_pattern)]

In [36]:
dataset_paths

['C:\\Users\\Zeki\\.keras\\datasets\\rockpaperscissors\\rps-cv-images\\paper\\04l5I8TqdzF9WDMJ.png',
 'C:\\Users\\Zeki\\.keras\\datasets\\rockpaperscissors\\rps-cv-images\\paper\\0a3UtNzl5Ll3sq8K.png',
 'C:\\Users\\Zeki\\.keras\\datasets\\rockpaperscissors\\rps-cv-images\\paper\\0cb6cVL8pkfi4wF6.png',
 'C:\\Users\\Zeki\\.keras\\datasets\\rockpaperscissors\\rps-cv-images\\paper\\0eqArS2GgsBeqgSn.png',
 'C:\\Users\\Zeki\\.keras\\datasets\\rockpaperscissors\\rps-cv-images\\paper\\0Og76sl5CJhbxWWx.png',
 'C:\\Users\\Zeki\\.keras\\datasets\\rockpaperscissors\\rps-cv-images\\paper\\0RA9rcrv8iVvuDDU.png',
 'C:\\Users\\Zeki\\.keras\\datasets\\rockpaperscissors\\rps-cv-images\\paper\\0t08v9bAaA0mXGCQ.png',
 'C:\\Users\\Zeki\\.keras\\datasets\\rockpaperscissors\\rps-cv-images\\paper\\0Uomd0HvOB33m47I.png',
 'C:\\Users\\Zeki\\.keras\\datasets\\rockpaperscissors\\rps-cv-images\\paper\\0vugygEjxQJPr9yz.png',
 'C:\\Users\\Zeki\\.keras\\datasets\\rockpaperscissors\\rps-cv-images\\paper\\0zKU6wEhT2cDe

In [37]:
train_paths, test_paths = train_test_split(dataset_paths,test_size=0.2,random_state=999)
train_paths, val_paths = train_test_split(train_paths,test_size=0.2,random_state=999)

In [41]:
print(len(train_paths))
print(len(val_paths))
print(len(test_paths))

1400
350
438


In [42]:
BATCH_SIZE = 1024
BUFFER_SIZE = 1024

train_dataset = prepare_dataset(train_paths,
                                buffer_size=BUFFER_SIZE,
                                batch_size=BATCH_SIZE)
validation_dataset = prepare_dataset(val_paths,
                                     buffer_size=BUFFER_SIZE,
                                     batch_size=BATCH_SIZE,
                                     shuffle=False)
test_dataset = prepare_dataset(test_paths,
                               buffer_size=BUFFER_SIZE,
                               batch_size=BATCH_SIZE,
                               shuffle=False)

In [43]:
train_dataset

<PrefetchDataset shapes: ((None, 32, 32, 1), (None, 3)), types: (tf.float32, tf.float32)>

In [44]:
validation_dataset

<PrefetchDataset shapes: ((None, 32, 32, 1), (None, 3)), types: (tf.float32, tf.float32)>

In [45]:
model = build_network()
model.compile(loss=CategoricalCrossentropy(from_logits=True),optimizer='adam',metrics=['accuracy'])

In [46]:
EPOCHS = 250
model.fit(train_dataset,validation_data=validation_dataset,epochs=EPOCHS)

Epoch 1/250
2/2 [==============================] - 4s 684ms/step - loss: 1.0978 - accuracy: 0.3621 - val_loss: 1.0880 - val_accuracy: 0.4743
Epoch 2/250
2/2 [==============================] - 1s 357ms/step - loss: 1.0918 - accuracy: 0.4429 - val_loss: 1.0815 - val_accuracy: 0.3457
Epoch 3/250
2/2 [==============================] - 1s 362ms/step - loss: 1.0769 - accuracy: 0.4071 - val_loss: 1.0753 - val_accuracy: 0.3829
Epoch 4/250
2/2 [==============================] - 1s 398ms/step - loss: 1.0591 - accuracy: 0.4750 - val_loss: 1.0555 - val_accuracy: 0.4629
Epoch 5/250
2/2 [==============================] - 1s 339ms/step - loss: 1.0530 - accuracy: 0.4900 - val_loss: 1.0506 - val_accuracy: 0.4943
Epoch 6/250
2/2 [==============================] - 1s 408ms/step - loss: 1.0340 - accuracy: 0.5564 - val_loss: 1.0377 - val_accuracy: 0.6171
Epoch 7/250
2/2 [==============================] - 1s 327ms/step - loss: 1.0172 - accuracy: 0.6450 - val_loss: 1.0251 - val_accuracy: 0.5314
Epoch 8/250
2

Epoch 59/250
2/2 [==============================] - 1s 430ms/step - loss: 0.7632 - accuracy: 0.8107 - val_loss: 0.7823 - val_accuracy: 0.8029
Epoch 60/250
2/2 [==============================] - 1s 423ms/step - loss: 0.7574 - accuracy: 0.8236 - val_loss: 0.7747 - val_accuracy: 0.8057
Epoch 61/250
2/2 [==============================] - 1s 448ms/step - loss: 0.7622 - accuracy: 0.8093 - val_loss: 0.7749 - val_accuracy: 0.8114
Epoch 62/250
2/2 [==============================] - 1s 309ms/step - loss: 0.7555 - accuracy: 0.8179 - val_loss: 0.7736 - val_accuracy: 0.8229
Epoch 63/250
2/2 [==============================] - 1s 419ms/step - loss: 0.7592 - accuracy: 0.8193 - val_loss: 0.7716 - val_accuracy: 0.8200
Epoch 64/250
2/2 [==============================] - 1s 363ms/step - loss: 0.7510 - accuracy: 0.8221 - val_loss: 0.7668 - val_accuracy: 0.8229
Epoch 65/250
2/2 [==============================] - 1s 437ms/step - loss: 0.7460 - accuracy: 0.8371 - val_loss: 0.7634 - val_accuracy: 0.8229
Epoch 

2/2 [==============================] - 1s 433ms/step - loss: 0.6722 - accuracy: 0.9100 - val_loss: 0.6985 - val_accuracy: 0.8771
Epoch 117/250
2/2 [==============================] - 1s 302ms/step - loss: 0.6704 - accuracy: 0.9100 - val_loss: 0.6977 - val_accuracy: 0.8743
Epoch 118/250
2/2 [==============================] - 1s 479ms/step - loss: 0.6690 - accuracy: 0.9186 - val_loss: 0.6967 - val_accuracy: 0.8771
Epoch 119/250
2/2 [==============================] - 1s 312ms/step - loss: 0.6679 - accuracy: 0.9150 - val_loss: 0.6967 - val_accuracy: 0.8829
Epoch 120/250
2/2 [==============================] - 1s 450ms/step - loss: 0.6655 - accuracy: 0.9179 - val_loss: 0.6952 - val_accuracy: 0.8800
Epoch 121/250
2/2 [==============================] - 1s 321ms/step - loss: 0.6645 - accuracy: 0.9193 - val_loss: 0.6942 - val_accuracy: 0.8829
Epoch 122/250
2/2 [==============================] - 1s 444ms/step - loss: 0.6666 - accuracy: 0.9150 - val_loss: 0.6944 - val_accuracy: 0.8829
Epoch 123/250

2/2 [==============================] - 1s 451ms/step - loss: 0.6224 - accuracy: 0.9571 - val_loss: 0.6686 - val_accuracy: 0.9000
Epoch 174/250
2/2 [==============================] - 1s 387ms/step - loss: 0.6201 - accuracy: 0.9586 - val_loss: 0.6701 - val_accuracy: 0.8971
Epoch 175/250
2/2 [==============================] - 1s 480ms/step - loss: 0.6195 - accuracy: 0.9614 - val_loss: 0.6669 - val_accuracy: 0.8971
Epoch 176/250
2/2 [==============================] - 1s 454ms/step - loss: 0.6204 - accuracy: 0.9557 - val_loss: 0.6685 - val_accuracy: 0.8943
Epoch 177/250
2/2 [==============================] - 1s 395ms/step - loss: 0.6201 - accuracy: 0.9579 - val_loss: 0.6661 - val_accuracy: 0.9029
Epoch 178/250
2/2 [==============================] - 1s 436ms/step - loss: 0.6173 - accuracy: 0.9629 - val_loss: 0.6664 - val_accuracy: 0.9029
Epoch 179/250
2/2 [==============================] - 1s 358ms/step - loss: 0.6192 - accuracy: 0.9607 - val_loss: 0.6659 - val_accuracy: 0.9029
Epoch 180/250

2/2 [==============================] - 1s 373ms/step - loss: 0.5940 - accuracy: 0.9779 - val_loss: 0.6508 - val_accuracy: 0.9171
Epoch 231/250
2/2 [==============================] - 1s 451ms/step - loss: 0.5911 - accuracy: 0.9807 - val_loss: 0.6514 - val_accuracy: 0.9229
Epoch 232/250
2/2 [==============================] - 1s 462ms/step - loss: 0.5927 - accuracy: 0.9800 - val_loss: 0.6522 - val_accuracy: 0.9143
Epoch 233/250
2/2 [==============================] - 1s 435ms/step - loss: 0.5913 - accuracy: 0.9821 - val_loss: 0.6520 - val_accuracy: 0.9171
Epoch 234/250
2/2 [==============================] - 1s 544ms/step - loss: 0.5919 - accuracy: 0.9786 - val_loss: 0.6499 - val_accuracy: 0.9171
Epoch 235/250
2/2 [==============================] - 1s 320ms/step - loss: 0.5919 - accuracy: 0.9807 - val_loss: 0.6512 - val_accuracy: 0.9229
Epoch 236/250
2/2 [==============================] - 1s 452ms/step - loss: 0.5928 - accuracy: 0.9786 - val_loss: 0.6510 - val_accuracy: 0.9171
Epoch 237/250

In [47]:
model.save('rockpaperscicors.hdf5')

In [48]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Loss: {test_loss}, accuracy: {test_accuracy}')

1/1 [==============================] - 1s 579ms/step - loss: 0.6412 - accuracy: 0.9224
Loss: 0.6412088871002197, accuracy: 0.922374427318573


In [50]:
#for one image prediction
image_path=r'C:\Users\Zeki\.keras\datasets\rockpaperscissors\rps-cv-images\scissors\3Q6h5cXYeNTeOLgD.png'
image = tf.io.read_file(image_path)
image = tf.image.decode_jpeg(image, channels=3)
image = tf.image.rgb_to_grayscale(image)
image = tf.image.convert_image_dtype(image, np.float32)
image = tf.image.resize(image, (32,32))

label = tf.strings.split(image_path, os.path.sep)[-2]
label = (label == CLASSES)  # One-hot encode.
label = tf.dtypes.cast(label, tf.float32)

In [60]:
from tensorflow.keras.preprocessing.image import *

In [65]:
img = load_img(image_path, target_size=(32, 32),
                         color_mode='grayscale')
img = img_to_array(img)

In [66]:
img.shape

(32, 32, 1)

In [68]:
test_image = np.expand_dims(img, axis=0)

In [75]:
test_image

array([[[[81.],
         [83.],
         [90.],
         ...,
         [87.],
         [87.],
         [86.]],

        [[86.],
         [88.],
         [88.],
         ...,
         [89.],
         [89.],
         [87.]],

        [[87.],
         [86.],
         [87.],
         ...,
         [93.],
         [91.],
         [89.]],

        ...,

        [[91.],
         [90.],
         [92.],
         ...,
         [96.],
         [93.],
         [95.]],

        [[90.],
         [91.],
         [93.],
         ...,
         [93.],
         [94.],
         [94.]],

        [[89.],
         [89.],
         [91.],
         ...,
         [93.],
         [93.],
         [90.]]]], dtype=float32)

In [70]:
test_image.shape

(1, 32, 32, 1)

In [72]:
test_image2= np.expand_dims(image, axis=0)

In [71]:
model.predict(test_image)

array([[0., 0., 1.]], dtype=float32)

In [73]:
test_image2.shape

(1, 32, 32, 1)

In [76]:
test_image2

array([[[[0.31786153],
         [0.32500002],
         [0.3456189 ],
         ...,
         [0.34166667],
         [0.3405331 ],
         [0.337163  ]],

        [[0.33829656],
         [0.33924633],
         [0.34120712],
         ...,
         [0.35042894],
         [0.3498162 ],
         [0.33676472]],

        [[0.3407782 ],
         [0.3422488 ],
         [0.3423713 ],
         ...,
         [0.35891545],
         [0.35713848],
         [0.34509805]],

        ...,

        [[0.3545037 ],
         [0.3504902 ],
         [0.3617341 ],
         ...,
         [0.37668508],
         [0.3642157 ],
         [0.36773896]],

        [[0.35058212],
         [0.35511643],
         [0.36516544],
         ...,
         [0.36516544],
         [0.368413  ],
         [0.3632353 ]],

        [[0.34482232],
         [0.34754902],
         [0.35716912],
         ...,
         [0.3647059 ],
         [0.36553308],
         [0.3505515 ]]]], dtype=float32)

In [74]:
model.predict(test_image2)

array([[2.8142691e-04, 9.8705757e-03, 9.8984796e-01]], dtype=float32)